In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monopoly-game-values/monopolygame.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
game_values = pd.read_csv("/kaggle/input/monopoly-game-values/monopolygame.csv")
game_values.head()

,name,type,color,spaces,cost,mortgage,house_price,rent_0,rent_1,rent_2,rent_3,rent_4,rent_hotel
0,Mediterranean,street,purple,1,60,30,50.0,2.0,10.0,30.0,90.0,160.0,250.0
1,Baltic,street,purple,3,60,30,50.0,4.0,20.0,60.0,180.0,320.0,450.0
2,Reading RR,railroad,NaN,5,200,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oriental,street,lightblue,6,100,50,50.0,6.0,30.0,90.0,270.0,400.0,550.0
4,Vermont,street,lightblue,8,100,50,50.0,6.0,30.0,90.0,270.0,400.0,550.0


In [4]:

# Function to compute the optimal houses and value
def calculate_optimal_value(row):
    if row["type"] != "street" or pd.isna(row["house_price"]):
        # Non-street properties or properties without house prices
        return pd.Series({"optimal_houses": 0, "optimal_value": row["rent_0"] or 0})
    
    # Calculate ROI for each house count
    rents = [row[f"rent_{i}"] for i in range(5)] + [row["rent_hotel"]]
    costs = [row["cost"] + i * row["house_price"] for i in range(6)]
    rois = [rent / cost if rent is not None else -np.inf for rent, cost in zip(rents, costs)]
    
    # Find optimal houses
    optimal_houses = np.argmax(rois)
    optimal_value = rents[optimal_houses]
    
    return pd.Series({"optimal_houses": optimal_houses, "optimal_value": optimal_value})

# Apply the function
game_values[["optimal_houses", "optimal_value"]] = game_values.apply(calculate_optimal_value, axis=1)

# Display the results
print(game_values[["name", "optimal_houses", "optimal_value"]])

                name  optimal_houses  optimal_value
0      Mediterranean             5.0          250.0
1             Baltic             5.0          450.0
2         Reading RR             0.0            NaN
3           Oriental             5.0          550.0
4            Vermont             5.0          550.0
5        Connecticut             5.0          600.0
6        St. Charles             5.0          750.0
7   Electric Company             0.0            NaN
8             States             5.0          750.0
9           Virginia             5.0          900.0
10   Pennsylvania RR             0.0            NaN
11         St. James             5.0          950.0
12         Tennessee             5.0          950.0
13          New York             5.0         1000.0
14          Kentucky             5.0         1050.0
15           Indiana             5.0         1050.0
16          Illinois             5.0         1100.0
17            B&O RR             0.0            NaN
18          

In [5]:
# Function to calculate property value
def calculate_property_value(row):
    # Assign the number of houses based on results above
    if row['color'] == "darkgreen":
        houses = 3  # Green properties
    elif row['name'] == "Park Place":
        houses = 4  # Park Place
    else:
        houses = 5  # All other properties

    # Determine the rent for the property based on the number of houses
    if houses == 3:
        rent = row['rent_3']  # Rent with 3 houses
    elif houses == 4:
        rent = row['rent_4']  # Rent with 4 houses
    else:
        rent = row['rent_hotel']  # Rent with 5 houses

    # Calculate the cost (initial cost + houses * house cost)
    cost = row['cost'] + (houses * row['house_price'])

    # Calculate the property value (Rent - Cost)
    value = rent - cost
    return value

# Apply the function to each row in the dataset
game_values['value'] = game_values.apply(calculate_property_value, axis=1)

# Display the properties with their calculated values
print(game_values[['name', 'color', 'value']])

                name      color  value
0      Mediterranean     purple  -60.0
1             Baltic     purple  140.0
2         Reading RR        NaN    NaN
3           Oriental  lightblue  200.0
4            Vermont  lightblue  200.0
5        Connecticut  lightblue  230.0
6        St. Charles     violet  110.0
7   Electric Company        NaN    NaN
8             States     violet  110.0
9           Virginia     violet  240.0
10   Pennsylvania RR        NaN    NaN
11         St. James     orange  270.0
12         Tennessee     orange  270.0
13          New York     orange  300.0
14          Kentucky        red   80.0
15           Indiana        red   80.0
16          Illinois        red  110.0
17            B&O RR        NaN    NaN
18          Atlantic     yellow  140.0
19           Ventnor     yellow  140.0
20       Water Works        NaN    NaN
21    Marvin Gardens     yellow  170.0
22           Pacific  darkgreen    0.0
23       N. Carolina  darkgreen    0.0
24      Pennsylvania  dar

In [6]:
import random
import pandas as pd

# Ensure 'mortgaged' column exists and is of boolean type
def validate_game_values(game_values):
    if "mortgaged" not in game_values.columns:
        game_values["mortgaged"] = False  # Initialize with False (or any default value)
    if "houses" not in game_values.columns:
        game_values["houses"] = 0
    if "owner" not in game_values.columns:
        game_values["owner"] = None  # Initialize owner column with None (no owner)
    return game_values

# Initialize players
def initialize_players():
    return [{"money": 1500, "position": 0, "properties": [], "color_sets": {}, "name": f"Player {_+1}", "moves": 0, "rolls": 0, "houses_bought": {}} for _ in range(4)]  # Track rolls and houses bought

# Roll two dice
def roll_dice():
    return random.randint(1, 6) + random.randint(1, 6)

def calculate_rent(property_row):
    houses = property_row["houses"]
    
    # Default rent values to 0 if missing
    rent_values = {
        0: property_row.get("rent_0", 0),
        1: property_row.get("rent_1", 0),
        2: property_row.get("rent_2", 0),
        3: property_row.get("rent_3", 0),
        4: property_row.get("rent_4", 0),
        5: property_row.get("rent_hotel", 0)
    }
    
    # Check if the rent value is NaN, and replace it with 0 if it is
    rent = rent_values.get(houses, 0)
    
    # Handle potential NaN values explicitly, in case rent is still NaN
    if pd.isna(rent):
        rent = 0  # Default to 0 if rent is NaN
        
    return rent


# Simulate a single turn with trade logic
def play_turn(player, game_values, players):
    dice_roll = roll_dice()
    player["rolls"] += 1  # Track the number of rolls
    initial_position = player["position"]
    player["position"] = (player["position"] + dice_roll) % 40

    # Check if player passed "Go"
    if player["position"] < initial_position:
        player["money"] += 200
    
    current_space = player["position"]
    property_row = game_values.loc[game_values["spaces"] == current_space]
    
    if property_row.empty:
        return  # Not a property space
    
    property_row = property_row.iloc[0]
    owner = property_row.get("owner", None)
    
    if pd.isna(owner):  # Property is unowned
        cost = property_row.get("cost", 0) or 0
        if player["money"] >= cost:
            player["money"] -= cost
            game_values.loc[property_row.name, "owner"] = player["name"]
            player["properties"].append(property_row["name"])
            color = property_row["color"]
            if color:
                player["color_sets"][color] = player["color_sets"].get(color, 0) + 1
    elif owner != player["name"]:  # Property is owned by another player
        rent = calculate_rent(property_row)
        owner_player = next((p for p in players if p["name"] == owner), None)
        if owner_player is None:
            return
        
        if player["money"] >= rent:
            player["money"] -= rent
            owner_player["money"] += rent
        else:
            handle_bankruptcy(player, owner_player, game_values, players, rent)

# Handle bankruptcy: Player cannot pay rent and needs to mortgage properties
def handle_bankruptcy(player, owner_player, game_values, players, rent):
    sell_houses_for_mortgage(player, game_values)
    
    if player["money"] < rent:
        mortgaged_money = 0
        for property_name in player["properties"]:
            property_row = game_values.loc[game_values["name"] == property_name]
            if property_row.empty:
                continue
            property_row = property_row.iloc[0]
            
            if property_row["mortgage"] > 0 and property_row["owner"] == player["name"] and not property_row["mortgaged"]:
                mortgaged_money += property_row["mortgage"]
                player["money"] += property_row["mortgage"]
                game_values.loc[game_values["name"] == property_name, "mortgaged"] = True
                
                if player["money"] >= rent:
                    break

    if player["money"] >= rent:
        player["money"] -= rent
        owner_player["money"] += rent
    else:
        handle_bankruptcy_final(player, owner_player, game_values, players)

# Final handling for bankruptcy: Remove player and transfer properties to owner
def handle_bankruptcy_final(player, owner_player, game_values, players):
    for property_name in player["properties"]:
        property_row = game_values.loc[game_values["name"] == property_name]
        if property_row.empty:
            continue
        property_row = property_row.iloc[0]
        if property_row['owner'] == player["name"]:
            game_values.loc[game_values["name"] == property_name, "owner"] = owner_player["name"]
            owner_player["properties"].append(property_name)

    players.remove(player)

# Function to check and validate mortgage values in game_values
def validate_mortgage_values(game_values):
    if "mortgage" not in game_values.columns:
        game_values["mortgage"] = 0
    return game_values

game_values = validate_mortgage_values(game_values)

game_values["houses"] = 0
game_values["owner"] = None

def buy_houses(player, game_values):
    for color, count in player["color_sets"].items():
        color_group = game_values[game_values["color"] == color]
        if count == len(color_group):
            for idx, prop in color_group.iterrows():
                if pd.notna(prop["house_price"]) and prop["houses"] < 5:
                    house_price = prop["house_price"]
                    max_houses = 5 - prop["houses"]
                    affordable_houses = min(player["money"] // house_price, max_houses)
                    if affordable_houses > 0:
                        affordable_houses = int(affordable_houses)
                        for _ in range(affordable_houses):
                            player["money"] -= house_price
                            game_values.at[idx, "houses"] += 1

# Function to sell houses for mortgaging
def sell_houses_for_mortgage(player, game_values):
    for property_name in player["properties"]:
        property_row = game_values.loc[game_values["name"] == property_name]
        if property_row.empty:
            continue
        property_row = property_row.iloc[0]
        houses_on_property = property_row["houses"]
        house_price = property_row["house_price"]
        
        if houses_on_property > 0:
            houses_to_sell = houses_on_property
            for _ in range(houses_to_sell):
                player["money"] += house_price // 2
                game_values.at[property_row.name, "houses"] -= 1

# Handle property mortgaging with default values
def mortgage_property(player, property_name, game_values):
    property_row = game_values.loc[game_values["name"] == property_name]
    if property_row.empty:
        return
    property_row = property_row.iloc[0]
    
    if property_row["mortgaged"]:
        return
    
    mortgage = property_row.get("mortgage", 0) or 0
    player["money"] += mortgage
    game_values.loc[game_values["name"] == property_name, "mortgaged"] = True
    player["properties"].remove(property_name)

# Main simulation function (with adjusted logic for multiple runs)

# Main simulation function (with adjusted logic for multiple runs)
def simulate_monopoly(game_values, num_simulations):
    for sim in range(num_simulations):
        # Reset the game state for each simulation
        game_values["owner"] = None  # Reset ownership
        game_values["houses"] = 0  # Reset house counts
        game_values["mortgaged"] = False  # Reset mortgages (optional, if necessary)
        
        players = initialize_players()  # Reinitialize players at the start of each simulation
        game_values = validate_game_values(game_values)  # Revalidate the game values

        max_turns = 1000
        turn_counter = 0

        while turn_counter < max_turns * len(players) and len([p for p in players if p["money"] > 0]) > 1:
            for player in players[:]:  # Iterate over a copy of the list to handle player removal
                if player["money"] > 0 and player["moves"] < 100:  # Stop game if a player reaches 100 moves
                    play_turn(player, game_values, players)
                    player["moves"] += 1  # Increment the move count
                    buy_houses(player, game_values)  # Ensure houses are bought after each turn
            turn_counter += 1

            # End game condition: If only one player is left with money
            if len([p for p in players if p["money"] > 0]) == 1:
                break  # End the game if there's only one player left

        # Find the winning player (the player with the most money)
        winning_player = max(players, key=lambda p: p["money"])

        # Final output: Only the winner and relevant data
        print(f"Simulation {sim + 1} - Winner: {winning_player['name']} with ${winning_player['money']}")
        print(f"{winning_player['name']} bought the following properties with houses:")
        for _, property_row in game_values.iterrows():
            if property_row['owner'] == winning_player['name']:
                houses = property_row['houses']
                print(f"- {property_row['name']} with {houses} house(s)")

        # Summary rolls and houses
        for player in players:
            print(f"{player['name']} rolled the dice {player['rolls']} times.")
            print(f"Properties bought by {player['name']}:")
            for prop, count in player["houses_bought"].items():
                print(f" - {prop}: {count} house(s)")



# Simulate the game for 50 runs
simulate_monopoly(game_values, num_simulations=50)


Simulation 1 - Winner: Player 3 with $8514.0
Player 3 bought the following properties with houses:
- Mediterranean with 0 house(s)
- Baltic with 0 house(s)
- Vermont with 0 house(s)
- Connecticut with 0 house(s)
- St. Charles with 0 house(s)
- Electric Company with 0 house(s)
- Virginia with 0 house(s)
- St. James with 0 house(s)
- Tennessee with 0 house(s)
- Kentucky with 5 house(s)
- Indiana with 5 house(s)
- Illinois with 5 house(s)
- B&O RR with 0 house(s)
- Atlantic with 0 house(s)
- Water Works with 0 house(s)
- Pacific with 0 house(s)
- Short Line RR with 0 house(s)
- Park Place with 0 house(s)
- Boardwalk with 0 house(s)
Player 2 rolled the dice 100 times.
Properties bought by Player 2:
Player 3 rolled the dice 100 times.
Properties bought by Player 3:
Simulation 2 - Winner: Player 3 with $3802.0
Player 3 bought the following properties with houses:
- Connecticut with 0 house(s)
- St. James with 0 house(s)
- New York with 0 house(s)
- Illinois with 0 house(s)
Player 1 rolled th

In [ ]:
import matplotlib.pyplot as plt

# Extract the means for space_0 to space_39
means = board_tracking_df.mean()
space_means = means[[f"space_{i}" for i in range(40)]]

# Plotting the bar chart
plt.figure(figsize=(12, 6))
space_means.plot(kind='bar', color='skyblue', edgecolor='black')

# Adding titles and labels
plt.title('Mean Landings per Space', fontsize=16)
plt.xlabel('Spaces (0-39)', fontsize=14)
plt.ylabel('Mean Landings', fontsize=14)
plt.xticks(ticks=range(40), labels=[f"Space {i}" for i in range(40)], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


